In [2]:
import findspark
findspark.init()

In [3]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-sql-kafka-0-10_2.11:2.3.1 pyspark-shell'

In [4]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [5]:
spark = SparkSession.builder \
.master("local[4]") \
.appName("ReadFromKafka") \
.config("spark.driver.memory","2g") \
.config("spark.executor.memory", "4g") \
.getOrCreate()

In [6]:
df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("subscribe", "deneme") \
  .load()

In [7]:
df2 = df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")

In [8]:
words = df2.select(explode(split(col("value"), " ")).alias("value"))

In [9]:
word_counts = words.groupBy("value").count().sort(desc("count"))

In [11]:
query = word_counts.writeStream \
      .outputMode("complete") \
      .format("console") \
      .start()

In [ ]:
query.awaitTermination()

In [ ]:
# kafka-console-producer terminaline geçilir ve elle mesaj yazılır

In [ ]:
# Sonuçlar jupyter console üzerinden gözlemlenir. Siyah cmd terminalden

In [ ]:
query.stop()